In [1]:
import os
from statistics import mean
import time
import gym
import pybullet_envs
from rex_gym.envs import rex_gym_env
import pybullet
import argparse
from tensorboardX import SummaryWriter
from queue import Queue
import numpy as np
import random
from tqdm import tqdm
import collections
import multiprocessing as mp
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
np.random.seed(43)
torch.manual_seed(43)


/home/anirudh/miniconda3/envs/work_env/lib/python3.10/site-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
pybullet build time: Sep  1 2022 13:08:43


##### ARGS

In [2]:
# ENV_NAME = "MinitaurBulletEnv-v0"
# ENV_NAME = "MinitaurTrottingEnv-v0"
BATCH_SIZE = 128
LR_AGENT = 1e-5
LR_CRITIC = 1e-5
BUFFER_MAXLEN = 100_000
REPLAY_INITIAL = 20_000
REPLAY_START_SIZE = 1_000
DEV = "cuda" if torch.cuda.is_available() else "cpu"
# print(DEV)
REPEAT_STEPS = 5
MAX_EPSILON = 1.0
MIN_EPSILON = 1e-2
GAMMA = 0.99
ENERGY_WEIGHT = 1e-4
MODEL_SAVE_PATH = "rex_models/"
HEIGHT = 0.15
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)


In [3]:

def getAnglesFromMatrix(rot_mat):
    O_x = np.arctan2(rot_mat[3*2+1], rot_mat[3*2+2])
    O_y = np.arctan2(-rot_mat[3*2], np.sqrt(rot_mat[3*2+1]**2+rot_mat[3*2+2]**2))
    O_z = np.arctan2(rot_mat[3*1], rot_mat[0])
    return (O_x, O_y, O_z)

def pos_n_ori(env):
    '''
    env : Gym env
    '''
    # current position and orientation
    pos = env.rex.GetBasePosition()
    # pos is 3-tuple [x,y,z]
    rot = env.rex.GetBaseOrientation()
    rot_mat = pybullet.getMatrixFromQuaternion(rot)
    rot_ang = getAnglesFromMatrix(rot_mat)
    # rot_ang is 3-tuple [alpha, beta, gamma]
    return pos, rot_ang


def get_reward(control, pos1, ori1, pos2, ori2, torque, vel):
    '''
    control : desired movement of the bot
        0 : front
        1 : right
        2 : back
        3 : left
        4 : turn cw
        5 : turn ccw
    '''
    del_x = pos2[0] - pos1[0]
    del_y = pos2[1] - pos1[1]
    err_z = (pos2[2] - HEIGHT) 
    del_d = np.sqrt(del_x**2 + del_y**2)
    # theta = np.arctan2(del_y, del_x)
    # del_theta = theta - ori1[2]
    del_alpha = ori2[0] - ori1[0]
    del_beta = ori2[1] - ori1[1]
    del_gamma = ori2[2] - ori1[2]
    # del_f = del_d * np.cos(del_theta)
    # del_l = del_d * np.sin(del_theta)
    # del_b = -del_f
    # del_r = -del_l
    energy_r = 0.
    r = 0.
    if ENERGY_WEIGHT > 0:
        for t, v in zip(torque, vel):
            energy_r += ENERGY_WEIGHT * np.abs(np.dot(t, v))
    r -= energy_r
    r -= 0e0 * np.abs(err_z) # penalize change in height
    if control == 0:
        # r = del_f - np.abs(del_l) - np.abs(del_gamma)
        # r = del_f
        r += 5e1 * del_x - 1e1*np.abs(del_y) - np.abs(del_gamma)
    elif control == 1:
        # r = del_r - np.abs(del_f) - np.abs(del_gamma)
        # r = del_r
        r += 5e1 * -del_y - np.abs(del_x) - np.abs(del_gamma) 
    elif control == 2:
        # r = del_b - np.abs(del_l) - np.abs(del_gamma)
        # r = del_b
        r += 5e1 * -del_x - 1e1*np.abs(del_y) - np.abs(del_gamma)
    elif control == 3:
        # r = del_l - np.abs(del_f) - np.abs(del_gamma)
        # r = del_l
        r += 5e1 * del_y - np.abs(del_x) - np.abs(del_gamma)
    elif control == 4:
        # r = -del_gamma - np.abs(del_f) - np.abs(del_l)
        r += 5e1 * -del_gamma - np.abs(del_d)
    elif control == 5:
        # r = del_gamma - np.abs(del_f) - np.abs(del_l)
        r += 5e1 * del_gamma - np.abs(del_d) 
    return r



#### Networks


In [4]:
from rex_networks import Agent_net, Critic_net

In [5]:


class Agent():
    def __init__(self, n_obs, n_act, dev=DEV, rep_steps=REPEAT_STEPS, batch_size=BATCH_SIZE, epsilon= MAX_EPSILON) -> None:
        ## REPLAY BUFFER
        # 1. always keep some old experiences in replay_buf_initial to prevent catastrophic forgetting.
        self.buff_maxlen = BUFFER_MAXLEN
        self.replay_buf_initial = collections.deque(maxlen = REPLAY_INITIAL)
        self.replay_buf= collections.deque(maxlen = self.buff_maxlen - REPLAY_INITIAL)

        ## NETWORKS
        self.action_net = None
        self.action_net_targ = None
        self.critic_net = None
        self.critic_net_targ = None
        self.actor_optim = None
        self.critic_optim = None
        self.init_nets(n_obs, n_act, dev)
        self.update_targ_nets(alpha = 0.)
        self.mse_loss = nn.MSELoss()

        ## EPISODE and STATE variables
        self.dev = dev
        self.objective = nn.MSELoss()
        self.batch_size = batch_size
        self.rep_steps = rep_steps
        self.curr_ep_r = 0.
        self.epsilon = epsilon
        self.last_pos_n_ori = None

    def add_transn(self, control, s, s_next, r, a, is_done):
        '''Add new transn tuple in replay buffer'''
        if len(self.replay_buf_initial) < self.replay_buf_initial.maxlen:
            self.replay_buf_initial.append((control, s, s_next, r, a, is_done))
        else:
            self.replay_buf.append((control, s, s_next, r, a, is_done))


    @torch.no_grad()
    def move_k_steps(self, s, env, control, ep_len = 1, k=None):
        s_new = s
        s_new = torch.Tensor(s_new).unsqueeze(0).float().to(self.dev)
        a = self.action_net(s_new).squeeze().cpu().numpy()
        a += self.epsilon * np.random.normal(size=a.shape)
        a = np.clip(a, -1, 1)
        torque, vel = [], []
        # print(ep_len)
        # if ep_len == 1:
        self.last_pos_n_ori = pos_n_ori(env)
        if not k:
            k = self.rep_steps
        r = 0
        for i in range(k):
            s_new, _r, is_done, _ = env.step(a)
            r += _r
            torque.append(env.rex.GetMotorTorques())
            vel.append(env.rex.GetMotorVelocities())

            if is_done: 
                break
        pos2, rot2 = pos_n_ori(env)
        # r = get_reward(control, self.last_pos_n_ori[0], self.last_pos_n_ori[1], pos2, rot2, torque, vel)
        # r *= np.log(4 * ep_len)
        self.curr_ep_r += r
        curr_ep_r = self.curr_ep_r
        if is_done:
            s_new = env.reset()
            self.curr_ep_r = 0.
        # self.last_pos_n_ori = pos2, rot2
        return s, s_new, r, a, is_done, curr_ep_r


    def get_batch(self, batch_size=BATCH_SIZE):
        def get_batch_from_buff(buff, batch_size, control, s, s_next, r, a, is_done):
            minibatch = random.sample(buff, batch_size)
            for sample in minibatch:
                control.append(sample[0])
                s.append(sample[1])
                s_next.append(sample[2])
                r.append(sample[3])
                a.append(sample[4])
                is_done.append(sample[5])
                
        control, s, s_next, r, a, is_done = [], [], [], [], [], [] 
        if len(self.replay_buf) > batch_size - batch_size//3:
            get_batch_from_buff( self.replay_buf_initial, batch_size // 3, control, s, s_next, r, a, is_done)
            get_batch_from_buff( self.replay_buf, batch_size - batch_size // 3, control, s, s_next, r, a, is_done)
        else:
            get_batch_from_buff(self.replay_buf_initial, batch_size, control, s, s_next, r, a, is_done)
            
        s = torch.FloatTensor(s).to(self.dev)
        s_next = torch.FloatTensor(s_next).to(self.dev)
        r = torch.FloatTensor(r).to(self.dev)
        a = torch.FloatTensor(a).to(self.dev)
        is_done = torch.BoolTensor(is_done).to(self.dev)
        return control, s, s_next, r, a, is_done
    

    def train_iter(self, batch_size=None, rep_steps=None):
        if len(self.replay_buf_initial) + len(self.replay_buf) < REPLAY_START_SIZE:
            return None, None

        if rep_steps is None:
            rep_steps = self.rep_steps
        if batch_size is None:
            batch_size = self.batch_size
        control, s, s_next, r, a, is_done = self.get_batch(batch_size)
        # print(s.shape, s_next.shape, r.shape, a.shape, is_done.shape)

        #CRITIC
        self.critic_optim.zero_grad()
        q_m = self.critic_net(s, a)
        a_next = self.action_net_targ(s_next)
        _q_t = self.critic_net_targ(s_next, a_next)
        _q_t[is_done] = 0.
        q_t = r.unsqueeze(dim=-1) + ((GAMMA**rep_steps) * _q_t)  
        loss_critic = self.mse_loss(q_m, q_t.detach())
        loss_critic.backward()
        self.critic_optim.step()

        # ACTOR
        self.actor_optim.zero_grad() 
        a_m = self.action_net(s)
        loss_actor = -self.critic_net(s, a_m).mean()
        loss_actor.backward()
        self.actor_optim.step()

        return loss_actor, loss_critic
        
    def save_net(self, epoch, path="."):
        model_state = {
            "epoch": epoch,
            "action_model": self.action_net.state_dict(),
            "action_optim": self.actor_optim.state_dict(),
            "critic_model": self.critic_net.state_dict(),
            "critic_optim": self.critic_optim.state_dict(),
        }
        path = f"{path}/model_{epoch}.pt"
        torch.save(model_state, path)


    @torch.no_grad()
    def update_targ_nets(self, alpha):
        def update_targ_net(net, targ_net):
            net_dict = net.state_dict()
            targ_net_dict = targ_net.state_dict()
            for k, v in net_dict.items():
                targ_net_dict[k] = targ_net_dict[k] * alpha + v * (1-alpha)
            targ_net.load_state_dict(targ_net_dict)

        update_targ_net(self.action_net, self.action_net_targ)
        update_targ_net(self.critic_net, self.critic_net_targ)
        

    def init_nets(self, n_obs, n_act, dev):
        self.action_net = Agent_net(n_obs, n_act, dev).to(dev)
        self.action_net_targ = Agent_net(n_obs, n_act, dev).to(dev).requires_grad_(False)
        self.critic_net = Critic_net(n_obs, n_act, dev).to(dev)
        self.critic_net_targ = Critic_net(n_obs, n_act, dev).to(dev).requires_grad_(False)
        self.actor_optim = optim.Adam(self.action_net.parameters(), lr=LR_AGENT)
        self.critic_optim = optim.Adam(self.critic_net.parameters(), lr=LR_CRITIC)

    def load_net(self, fname):
        model_state = torch.load(fname, map_location=self.dev)
        self.net.load_state_dict(model_state['model'])
        self.optimizer.load_state_dict(model_state['optim'])
        return model_state["epoch"]




## Training Loop

In [6]:
if __name__ == "__main__":
    writer = SummaryWriter(comment=f"rex_ddpg")
    env = rex_gym_env.RexGymEnv(hard_reset=False, render=False, terrain_id="plane")
    # print(env.observation_space.shape)
    # print(env.action_space.shape)
    agent = Agent(
        env.observation_space.shape[0],
        env.action_space.shape[0],
    )
    s = env.reset()
    pbar = tqdm()
    i = 0
    


/home/anirudh/miniconda3/envs/work_env/lib/python3.10/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
0it [00:00, ?it/s]

In [7]:
    ep_len = 0
    last_ep_r = 0.
    last_ep_len = 0
    best_r = None
    last_1_r = collections.deque(maxlen=1)
    sum_r = 0.

    control = 0
    random_control = False
    print(f"\n##################################\nCONTROL : {control :1d} random_control : {random_control} \n##################################")

    while(True):
        i += 1
        ep_len += 1
        if random_control and ep_len == 1:
            control = random.choice([0,2,4,5])
        # gather exp
        s, s_next, r, a, is_done, ep_r = agent.move_k_steps(s, env, control, ep_len, k=REPEAT_STEPS)
        pos = env.rex.GetBasePosition()
        agent.add_transn(control, s, s_next, r, a, is_done)
        s = s_next
        if (pos[2] < 0.07):
            r = 0
            s = env.reset()
        if is_done:
            last_ep_r = ep_r
            last_ep_len = ep_len
            ep_len = 0
            if len(last_1_r) == last_1_r.maxlen:
                sum_r -= last_1_r.popleft() 
            sum_r += last_ep_r
            last_1_r.append(last_ep_r)
            # print("average r", sum_r / len(last_10_r), "best_r", best_r)
            if best_r is None or (sum_r / len(last_1_r)) > best_r:
                best_r = sum_r / len(last_1_r) 
                if random_control:
                    agent.save_net("all")
                else:
                    agent.save_net(control, path=MODEL_SAVE_PATH)
        # train
        loss_agent, loss_critic = agent.train_iter(batch_size=BATCH_SIZE, rep_steps=REPEAT_STEPS)
        agent.update_targ_nets(alpha=1-1e-3)
        agent.epsilon = MAX_EPSILON - (MAX_EPSILON - MIN_EPSILON) * (i / 15_000)
        agent.epsilon = max(agent.epsilon, MIN_EPSILON)
        
        if i % 20 == 0:
            pbar.update(20)
            if loss_agent is not None:
                pbar.set_description(f"last ep length : {last_ep_len}, last episode reward : {last_ep_r:.4f}, loss agent = {loss_agent.item():.4f}, loss critic = {loss_critic.item():.4f}")
                writer.add_scalar("episode length", last_ep_len, i)
                writer.add_scalar("episode reward", last_ep_r, i)
                writer.add_scalar("loss agent",loss_agent.item(), i)
                writer.add_scalar("loss critic", loss_critic.item(), i)
                writer.add_scalar("epsilon", agent.epsilon, i)

    # %%



##################################
CONTROL : 0 random_control : False 
##################################


20it [00:01, 11.92it/s]

(0.0, 0.0, 0.21)
(0.0, 0.0, 0.21)
(0.009766766399744907, -0.001672462635555105, 0.2652693724799243)
(0.007946560669962837, 0.005613027530051241, 0.33363090776962184)
(-0.004572219830124406, -0.005653176809247399, 0.36963620183055296)
(0.0, 0.0, 0.21)
(0.001413173076108241, -0.0083982948019182, 0.26990013877744573)
(0.0, 0.0, 0.21)
(0.0037248413942950337, -0.006338562467152888, 0.2594659613328984)
(0.0011452321650586579, -0.027674660108420297, 0.29592086425117614)
(0.0, 0.0, 0.21)
(0.0046092267941735005, -0.0002459684029308978, 0.250385544844488)
(-0.0021602523617143625, 0.00011448465164990492, 0.2914552882918358)
(0.0, 0.0, 0.21)
(0.00616393288041976, -0.0018393890036913468, 0.2722493019801071)
(0.007260054505591633, -0.0028924091267146386, 0.3092699063119141)
(0.004503208789512165, 0.007503795123427879, 0.3476511866285975)
(0.005809716644878288, 0.010172634538758366, 0.3451721042500632)
(0.0037440189845171943, 0.019826965774449264, 0.30347254112992655)
(-0.004089467142126772, 0.021767

40it [00:01, 25.15it/s]

(0.004592875303870124, 0.027962701262532325, 0.22788422949805928)
(0.0, 0.0, 0.21)
(0.001965398393086698, -0.0003121547126848016, 0.250623353724684)
(0.00596597565125004, -0.00675188525187538, 0.29130792888410156)
(0.01036198839659092, -0.0034445758212715057, 0.32354374794599056)
(0.0, 0.0, 0.21)
(0.0004814439239829453, -0.002478777977624176, 0.2709181634031615)
(0.0, 0.0, 0.21)
(0.002357421543725269, -0.007291890191345087, 0.2507297070253015)
(0.0, 0.0, 0.21)
(-0.003131391315960241, 0.002849593450821189, 0.22569672904639906)
(-0.011577510906572335, 6.406321436300505e-05, 0.23204669015777107)
(0.0, 0.0, 0.21)
(0.0012564348684664243, -0.010971399634289622, 0.24148513240438566)
(0.0017041520591001775, -0.01609564821695736, 0.2388277580460243)
(-0.006294306155284049, -0.03796679449698497, 0.25979974417112506)
(0.0, 0.0, 0.21)
(0.0074835605052153, -0.008665339233497845, 0.26995843529536145)
(0.0, 0.0, 0.21)
(0.0017181303568461971, -0.003313027205699138, 0.24453241268802986)
(0.0, 0.0, 0.21

...

340it [00:04, 129.38it/s]

(0.0, 0.0, 0.21)
(0.007685384668649905, -0.00030387197769647813, 0.2559877966399449)
(0.0035854130606040426, 0.00023206058655468337, 0.3309923680662866)
(-0.006901389903434227, 0.012561813809694877, 0.35314356685910697)
(-0.007673978019169627, 0.02592179695969895, 0.3427053147087756)
(0.0, 0.0, 0.21)
(0.0046888940320689194, 0.0032790652226717147, 0.26287066409827947)
(0.0017295734610118614, -0.00034137701150628325, 0.2894090686716177)
(0.0, 0.0, 0.21)
(0.011292132933963816, 0.00938795985749021, 0.2678205232518922)
(0.0012307772123241276, 0.018475359303212317, 0.33039673328035407)
(0.0022925884421814468, 0.01964573800800224, 0.3608510958064553)
(-0.002521569666600218, 0.039240772390064016, 0.3734877321456759)
(-0.012672875720816481, 0.04334353574031596, 0.3743898754658627)
(0.0, 0.0, 0.21)
(0.005319415539300542, 0.0008743402944830297, 0.2712654322408706)
(0.0, 0.0, 0.21)
(-0.0007338535784101342, 0.008225005039187994, 0.2320982392415632)
(0.0, 0.0, 0.21)
(0.0, 0.0, 0.21)
(0.0, 0.0, 0.21)

380it [00:04, 134.54it/s]

(-0.018074791990921005, -0.04838615266719762, 0.29970230088895944)
(-0.026401028193549475, -0.07425415422445317, 0.2693347326789584)
(-0.031634027638692, -0.09126416747596114, 0.2537816997525826)
(0.0, 0.0, 0.21)
(0.00472571390708134, 0.0064703380670775665, 0.2532919965384827)
(0.004416437503660389, 0.014614624290881324, 0.29715348887822024)
(-0.0067856542618591345, 0.016091134431788497, 0.3057226844893451)
(-0.00912668396089662, 0.034515914375088254, 0.31845532380148994)
(-0.02411994153915615, 0.03246235813304288, 0.2996684346874133)
(-0.016747160302854402, 0.035588517042281165, 0.2990692071150671)
(0.0, 0.0, 0.21)
(0.0, 0.0, 0.21)
(0.0046318418233883335, -0.009344488836721218, 0.2574478987746161)
(0.0, 0.0, 0.21)
(-0.0009347568306549953, -0.010832724596782126, 0.2534281225610509)
(-0.0014999480934310006, -0.013298432836440943, 0.2974354780913466)
(-0.007134766407933444, -0.03253525030825083, 0.29721937773113805)
(0.0, 0.0, 0.21)
(0.0004096684588185482, 0.000660384095235696, 0.2492103

KeyboardInterrupt: 